# Azure AI Agents with Model Context Protocol (MCP) Support - Python

यह नोटबुक दिखाती है कि कैसे Azure AI Agents को Model Context Protocol (MCP) टूल्स के साथ Python में उपयोग किया जा सकता है। यह बताती है कि कैसे एक बुद्धिमान एजेंट बनाया जा सकता है जो बाहरी MCP सर्वर्स (जैसे Microsoft Learn) का उपयोग करके बिना कुंजी प्रमाणीकरण के उन्नत क्षमताओं का लाभ उठा सकता है।


## आवश्यक पायथन पैकेज इंस्टॉल करें

सबसे पहले, हमें आवश्यक पायथन पैकेज इंस्टॉल करने होंगे:
- **azure-ai-projects**: कोर Azure AI Projects SDK
- **azure-ai-agents**: एजेंट्स बनाने और प्रबंधित करने के लिए Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredential का उपयोग करके कुंजी रहित प्रमाणीकरण प्रदान करता है
- **mcp**: पायथन के लिए Model Context Protocol का कार्यान्वयन


## कीलेस प्रमाणीकरण के लाभ

यह नोटबुक **कीलेस प्रमाणीकरण** को प्रदर्शित करता है, जो कई फायदे प्रदान करता है:
- ✅ **एपीआई कुंजियों को प्रबंधित करने की आवश्यकता नहीं** - Azure पहचान-आधारित प्रमाणीकरण का उपयोग करता है
- ✅ **सुरक्षा में सुधार** - कोड या कॉन्फ़िगरेशन फ़ाइलों में कोई गुप्त जानकारी संग्रहीत नहीं होती
- ✅ **स्वचालित क्रेडेंशियल रोटेशन** - Azure क्रेडेंशियल जीवनचक्र प्रबंधन को संभालता है
- ✅ **भूमिका-आधारित पहुंच नियंत्रण** - सटीक अनुमतियों के लिए Azure RBAC का उपयोग करता है
- ✅ **मल्टी-एनवायरनमेंट समर्थन** - विकास और उत्पादन में सहजता से काम करता है

`DefaultAzureCredential` स्वचालित रूप से सबसे उपयुक्त क्रेडेंशियल स्रोत का चयन करता है:
1. **प्रबंधित पहचान** (जब Azure में चल रहा हो)
2. **Azure CLI** क्रेडेंशियल्स (स्थानीय विकास के दौरान)
3. **Visual Studio** क्रेडेंशियल्स
4. **पर्यावरणीय वेरिएबल्स** (यदि कॉन्फ़िगर किए गए हों)
5. **इंटरएक्टिव ब्राउज़र** प्रमाणीकरण (फॉलबैक के रूप में)


## कीलेस प्रमाणीकरण सेटअप

**कीलेस प्रमाणीकरण के लिए आवश्यकताएँ:**

### स्थानीय विकास के लिए:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure वातावरण के लिए:
- अपने Azure संसाधन पर **सिस्टम-असाइनड मैनेज्ड आइडेंटिटी** सक्षम करें
- मैनेज्ड आइडेंटिटी को उपयुक्त **RBAC भूमिकाएँ** असाइन करें:
  - Azure OpenAI एक्सेस के लिए `Cognitive Services OpenAI User`
  - Azure AI Projects एक्सेस के लिए `AI Developer`

### पर्यावरण वेरिएबल्स (वैकल्पिक):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**कोई API कुंजी या कनेक्शन स्ट्रिंग की आवश्यकता नहीं!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## आवश्यक लाइब्रेरी आयात करें

आवश्यक Python मॉड्यूल आयात करें:
- **os, time**: पर्यावरण वेरिएबल्स और देरी के लिए मानक Python लाइब्रेरी
- **AIProjectClient**: Azure AI Projects के लिए मुख्य क्लाइंट
- **DefaultAzureCredential**: Azure सेवाओं के लिए बिना कुंजी के प्रमाणीकरण
- **MCP-संबंधित क्लासेस**: MCP टूल्स बनाने और प्रबंधन करने तथा अनुमोदन संभालने के लिए


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP सर्वर सेटिंग्स कॉन्फ़िगर करें

पर्यावरण वेरिएबल्स के साथ डिफ़ॉल्ट विकल्पों का उपयोग करके MCP सर्वर कॉन्फ़िगरेशन सेट करें:
- **MCP_SERVER_URL**: MCP सर्वर का URL (डिफ़ॉल्ट Microsoft Learn API है)
- **MCP_SERVER_LABEL**: MCP सर्वर की पहचान के लिए एक लेबल (डिफ़ॉल्ट "mslearn" है)

यह तरीका विभिन्न वातावरणों में लचीली कॉन्फ़िगरेशन की अनुमति देता है।


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI प्रोजेक्ट क्लाइंट बनाएं (कीलेस ऑथेंटिकेशन)

**कीलेस ऑथेंटिकेशन** का उपयोग करके Azure AI प्रोजेक्ट क्लाइंट को इनिशियलाइज़ करें:
- **endpoint**: Azure AI Foundry प्रोजेक्ट का endpoint URL
- **credential**: सुरक्षित और कीलेस ऑथेंटिकेशन के लिए `DefaultAzureCredential()` का उपयोग करता है
- **API कुंजियों की आवश्यकता नहीं**: स्वचालित रूप से सबसे उपयुक्त क्रेडेंशियल का पता लगाता है और उपयोग करता है

**ऑथेंटिकेशन फ्लो:**
1. मैनेज्ड आइडेंटिटी की जांच करता है (Azure वातावरण में)
2. Azure CLI क्रेडेंशियल्स पर वापस जाता है (स्थानीय विकास के लिए)
3. आवश्यकता अनुसार अन्य उपलब्ध क्रेडेंशियल स्रोतों का उपयोग करता है

यह तरीका आपके कोड में API कुंजियों या कनेक्शन स्ट्रिंग्स को प्रबंधित करने की आवश्यकता को समाप्त करता है।


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP टूल परिभाषा बनाएं

एक MCP टूल बनाएं जो Microsoft Learn MCP सर्वर से जुड़ता है:
- **server_label**: MCP सर्वर के लिए पहचानकर्ता
- **server_url**: MCP सर्वर का URL एंडपॉइंट
- **allowed_tools**: वैकल्पिक सूची जो यह सीमित करती है कि कौन से टूल उपयोग किए जा सकते हैं (खाली सूची सभी टूल की अनुमति देती है)

यह टूल एजेंट को Microsoft Learn दस्तावेज़ और संसाधनों तक पहुंचने में सक्षम बनाएगा।


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## एजेंट बनाएं और बातचीत को निष्पादित करें (कीलेस वर्कफ़्लो)

यह व्यापक अनुभाग **कीलेस एजेंट वर्कफ़्लो** को पूरी तरह से समझाता है:

1. **एआई एजेंट बनाएं**: GPT-4.1 नैनो मॉडल और MCP टूल्स के साथ एक एजेंट सेट करें  
2. **थ्रेड बनाएं**: संवाद के लिए एक बातचीत थ्रेड स्थापित करें  
3. **संदेश भेजें**: एजेंट से Azure OpenAI और OpenAI के बीच के अंतर के बारे में पूछें  
4. **टूल अनुमोदन संभालें**: जब आवश्यक हो, MCP टूल कॉल्स को स्वचालित रूप से स्वीकृत करें  
5. **निष्पादन की निगरानी करें**: एजेंट की प्रगति को ट्रैक करें और आवश्यक कार्यों को संभालें  
6. **परिणाम प्रदर्शित करें**: बातचीत और टूल उपयोग का विवरण दिखाएं  

**कीलेस विशेषताएं:**
- ✅ **कोई हार्डकोडेड सीक्रेट्स नहीं** - सभी प्रमाणीकरण Azure पहचान द्वारा संभाला जाता है  
- ✅ **डिफ़ॉल्ट रूप से सुरक्षित** - भूमिका-आधारित एक्सेस नियंत्रण का उपयोग करता है  
- ✅ **सरल परिनियोजन** - क्रेडेंशियल प्रबंधन की आवश्यकता नहीं  
- ✅ **ऑडिट-फ्रेंडली** - सभी एक्सेस Azure पहचान के माध्यम से ट्रैक किए जाते हैं  

एजेंट MCP टूल्स का उपयोग करके Microsoft Learn संसाधनों तक पूरी सुरक्षा के साथ और बिना API कुंजी प्रबंधन के पहुंच बनाएगा।  


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
